In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
import torch.utils.data as data
from transformers import AutoTokenizer
import math
import copy
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
import time
from tqdm import tqdm
# 데이터셋 로드
dataset = load_dataset("bentrevett/multi30k", split='train', cache_dir="../../../data2/jeongseokoh/data/")
val_dataset = load_dataset("bentrevett/multi30k", split='validation', cache_dir="../../../data2/jeongseokoh/data/")

# Hugging Face의 토크나이저 로드 (예: BERT의 토크나이저)
tokenizer_src = AutoTokenizer.from_pretrained("bert-base-german-cased")
tokenizer_tgt = AutoTokenizer.from_pretrained("bert-base-uncased")

# 토큰화 함수
def tokenize_function(examples):
    # 영어(en)와 독일어(de) 텍스트에 대한 토크나이징 수행
    tokenized_inputs = tokenizer_src(examples["de"], padding="max_length", truncation=True, max_length=128)
    # 타겟 토크나이저로 영어 텍스트 토크나이징
    with tokenizer_tgt.as_target_tokenizer():
        tokenized_targets = tokenizer_tgt(examples["en"], padding="max_length", truncation=True, max_length=128)
    
    # 모델 입력을 위한 tokenized_inputs 반환 및 레이블(labels)로 tokenized_targets["input_ids"] 설정
    tokenized_inputs["labels"] = tokenized_targets["input_ids"]
    return tokenized_inputs
def collate_fn(batch):
    input_ids = torch.tensor([item['input_ids'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch])
    return {"input_ids": input_ids, "labels": labels}
# 데이터셋에 토큰화 함수 적용
tokenized_datasets = dataset.map(tokenize_function, batched=True)
val_tokenized_datasets = val_dataset.map(tokenize_function, batched=True)
# DataLoader 재정의
dataloader = DataLoader(tokenized_datasets, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_tokenized_datasets, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        #print(f"d_model : {d_model}")
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        #print(mask)
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        #print((x + self.pe[:, :x.size(1)]).shape)
        return x + self.pe[:, :x.size(1)]

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2).to(device)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3).to(device)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
src_vocab_size = tokenizer_ko.vocab_size
tgt_vocab_size = tokenizer_tgt.vocab_size
d_model = 768  # BERT의 경우
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 128
dropout = 0.1

# 모델 초기화
transformer = Transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    d_ff=d_ff,
    max_seq_length=max_seq_length,
    dropout=dropout
).to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_tgt.pad_token_id)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

# 모델 학습
transformer.train()

start_time = time.time()

for epoch in range(1, 11):  # 에폭 수를 조정할 수 있습니다.
    total_loss = 0
    for batch in tqdm(ko_dataloader, desc="Train Iteration"):
        src = batch['input_ids'].to(device)  # 장치로 이동
        tgt = batch['labels'].to(device)     # 장치로 이동

        optimizer.zero_grad()
        output = transformer(src, tgt[:, :-1])
        loss = criterion(output.reshape(-1, tgt_vocab_size), tgt[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch: {epoch}, Avg Train Loss: {avg_loss:.4f}")
    for batch in tqdm(ko_val_dataloader, desc="Val Iteration"):
        src = batch['input_ids'].to(device)  # 장치로 이동
        tgt = batch['labels'].to(device)     # 장치로 이동

        with torch.no_grad():
            output = transformer(src, tgt[:, :-1])
            loss = criterion(output.reshape(-1, tgt_vocab_size), tgt[:, 1:].reshape(-1))
        
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch: {epoch}, Avg Validation Loss: {avg_loss:.4f}")
end_time = time.time()
print(f"총 시간: {end_time - start_time} sec")